<a href="https://colab.research.google.com/github/azam-tents/Road-and-Car-Safety-Model-Testing-Repository/blob/main/yolov8_aerial%20vehicle_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U gdown


# Train Set Preparation

In [5]:
!gdown --folder https://drive.google.com/drive/folders/1gmeE3D7R62UAtuIFOB9j2M5cUPTwtsxK


Retrieving folder contents
Retrieving folder 1MvSH7sNaY4p4lhwAU_BG3y7zth6-rtrD images
Retrieving folder 1HVERtwwLpR7HX0kzRO6JuFn91bf42HTd 1
Processing file 1zb_kEOXTtpsMuWAtuTtMIPLVUSpjGPPw part1.zip
Processing file 1BlaGYNNEKGmT6OjZjsJ8HoUYrTTmFcO2 part1.zip
Processing file 1pEmwJtugIWhiwgBqOtplNUtTG2T454zn part3.zip
Processing file 1JBWCHdyZOd9ULX0ng5C9haAt3FMPXa3v part2.zip
Retrieving folder 1_ObTxq-IhRL-N4Ci9HTR7upxNTBTzaRL labelTxt-v1.5
Processing file 12uPWoADKggo9HGaqGh2qOmcXXn-zKjeX DOTA-v1.5_train.zip
Processing file 1-vLCMhIW9CV2cmCPPBbDR9_hdecf5bLb DOTA-v1.5_train_hbb.zip
Retrieving folder 1cCU7Mxs2YqQ26UBJcQklB1ZI6bv5fCt4 labelTxt-v1.0
Processing file 1I-faCP-DOxf6mxcjUTc8mYVPqUgSQxx6 labelTxt.zip
Processing file 1sS9hveKtYAiTsGVxC4msF5qJjhn3wYpY Train_Task2_gt.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1zb_kEOXTtpsMuWAtuTtMIPLVUSpjGPPw
From (redi

In [6]:
!ls train


images	labelTxt-v1.0  labelTxt-v1.5


In [7]:
import zipfile
import os

# List of zip files to extract
zip_files = ['part1.zip', 'part2.zip', 'part3.zip']

# Base paths
zip_base_path = 'train/images'
extract_dir = 'train/yolov8/images_all'

# Make sure output directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_name in zip_files:
    zip_path = os.path.join(zip_base_path, zip_name)
    print(f"🔄 Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print(f"✅ All zip files extracted to '{extract_dir}'")


🔄 Extracting train/images/part1.zip...
🔄 Extracting train/images/part2.zip...
🔄 Extracting train/images/part3.zip...
✅ All zip files extracted to 'train/yolov8/images_all'


In [8]:
import zipfile
import os

# Define source directories and destination
label_dirs = ['train/labelTxt-v1.0', 'train/labelTxt-v1.5']
output_dir = 'train/yolov8/labels'

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Loop through both label directories
for label_dir in label_dirs:
    for file in os.listdir(label_dir):
        if file.endswith('.zip'):
            zip_path = os.path.join(label_dir, file)
            print(f"🔄 Extracting {zip_path}...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)

print(f"✅ All label zips extracted to '{output_dir}'")


🔄 Extracting train/labelTxt-v1.0/labelTxt.zip...
🔄 Extracting train/labelTxt-v1.0/Train_Task2_gt.zip...
🔄 Extracting train/labelTxt-v1.5/DOTA-v1.5_train.zip...
🔄 Extracting train/labelTxt-v1.5/DOTA-v1.5_train_hbb.zip...
✅ All label zips extracted to 'train/yolov8/labels'


In [9]:
import os
from PIL import Image

# Class map (from dataset.yaml)
class_map = {
    'plane': 0, 'ship': 1, 'storage-tank': 2, 'baseball-diamond': 3,
    'tennis-court': 4, 'basketball-court': 5, 'ground-track-field': 6, 'harbor': 7,
    'bridge': 8, 'large-vehicle': 9, 'small-vehicle': 10, 'helicopter': 11,
    'roundabout': 12, 'soccer-ball-field': 13, 'swimming-pool': 14, 'container-crane': 15
}

# Paths
input_label_folder = '/content/train/yolov8/labels/trainset_reclabelTxt'
image_folder = '/content/train/yolov8/images_all/images'
output_label_folder = '/content/train/yolov8/labels/labels_yolo'
os.makedirs(output_label_folder, exist_ok=True)

image_exts = ['.jpg', '.jpeg', '.png']

def find_image_path(label_filename):
    base_name, _ = os.path.splitext(label_filename)
    for ext in image_exts:
        image_path = os.path.join(image_folder, base_name + ext)
        if os.path.exists(image_path):
            return image_path
    return None

def extract_yolo_from_polygon(coords, img_w, img_h):
    xs = coords[::2]
    ys = coords[1::2]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)

    cx = ((xmin + xmax) / 2) / img_w
    cy = ((ymin + ymax) / 2) / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h

    # Clamp values between 0 and 1
    return tuple(min(max(v, 0), 1) for v in (cx, cy, w, h))

# Process label files
for label_file in os.listdir(input_label_folder):
    if not label_file.endswith('.txt'):
        continue

    image_path = find_image_path(label_file)
    if not image_path:
        print(f"⚠️ Image not found for {label_file}. Skipping.")
        continue

    try:
        with Image.open(image_path) as img:
            img_w, img_h = img.size
    except Exception as e:
        print(f"❌ Failed to open image {image_path}: {e}")
        continue

    input_path = os.path.join(input_label_folder, label_file)
    output_path = os.path.join(output_label_folder, label_file)

    with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            parts = line.strip().split()
            # Your format has 10 parts: 8 coords + class_name + "0"
            if len(parts) != 10:
                print(f"⚠️ Malformed line in {label_file}: {line.strip()}")
                continue

            try:
                coords = list(map(float, parts[:8]))
                class_name = parts[8].lower().replace('_', '-').replace(' ', '-')
                # The last part (parts[9]) seems to be a trailing zero, ignore or validate if needed

                if class_name in class_map:
                    class_id = class_map[class_name]
                else:
                    print(f"❌ Unknown class '{class_name}' in {label_file}")
                    continue

                cx, cy, w, h = extract_yolo_from_polygon(coords, img_w, img_h)
                if w == 0 or h == 0:
                    print(f"⚠️ Zero-area box in {label_file}: {line.strip()}")
                    continue

                outfile.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

            except Exception as e:
                print(f"❌ Error processing line in {label_file}: {line.strip()} — {e}")


In [10]:
import os

output_folder = '/content/train/yolov8/labels/labels_yolo'

for filename in os.listdir(output_folder):
    if not filename.endswith('.txt'):
        continue
    with open(os.path.join(output_folder, filename), 'r') as f:
        for line_num, line in enumerate(f, 1):
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Malformed line {line_num} in {filename}: {line.strip()}")
                continue
            class_id, x_c, y_c, w, h = parts
            x_c, y_c, w, h = map(float, [x_c, y_c, w, h])
            if not (0 <= x_c <= 1 and 0 <= y_c <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                print(f"Out of range values at line {line_num} in {filename}: {x_c}, {y_c}, {w}, {h}")

print("✅ Normalization check complete.")


✅ Normalization check complete.


# Validation Set Preparation

In [11]:
!gdown --folder https://drive.google.com/drive/folders/1n5w45suVOyaqY84hltJhIZdtVFD9B224



Retrieving folder contents
Retrieving folder 1RV7Z5MM4nJJJPUs6m9wsxDOJxX6HmQqZ images
Processing file 1uCCCFhFQOJLfjBpcL5MC0DHJ9lgOaXWP part1.zip
Retrieving folder 1_ncfVzGon8br8ZKjo-HVzkL_ptEzwj8L labelTxt-v1.5
Processing file 1XDWNx3FkH9layL8jVUkEHJ_-CY8K4zse DOTA-v1.5_val_hbb.zip
Processing file 1FkCSOCy4ieNg1UZj1-Irfw6-Jgqa37cC DOTA-v1.5_val.zip
Retrieving folder 1nySK57mC4mEgblwCXW4BRyK2voly13LM labelTxt-v1.0
Processing file 1uFwxA4B7H8zcI1oD11bj0U8z88qroMlG labelTxt.zip
Processing file 1roMkDBK9753uS5tCmtYlRTyzrObjjJ83 Val_Task2_gt.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1uCCCFhFQOJLfjBpcL5MC0DHJ9lgOaXWP
From (redirected): https://drive.google.com/uc?id=1uCCCFhFQOJLfjBpcL5MC0DHJ9lgOaXWP&confirm=t&uuid=c4a6f3e6-b05a-416f-8fde-4877139145cf
To: /content/val/images/part1.zip
100% 3.34G/3.34G [00:28<00:00, 115MB/s]
Downloading...
From: https://drive.googl

In [12]:
!ls val


images	labelTxt-v1.0  labelTxt-v1.5


In [13]:
import zipfile
import os

# List of zip files to extract
zip_files = ['part1.zip']

# Base paths
zip_base_path = '/content/val/images'
extract_dir = 'val/yolov8/images_all'

# Make sure output directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_name in zip_files:
    zip_path = os.path.join(zip_base_path, zip_name)
    print(f"🔄 Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print(f"✅ All zip files extracted to '{extract_dir}'")


🔄 Extracting /content/val/images/part1.zip...
✅ All zip files extracted to 'val/yolov8/images_all'


In [14]:
import zipfile
import os

# Define source directories and destination
label_dirs = ['/content/val/labelTxt-v1.0', '/content/val/labelTxt-v1.5']
output_dir = 'val/yolov8/labels'

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Loop through both label directories
for label_dir in label_dirs:
    for file in os.listdir(label_dir):
        if file.endswith('.zip'):
            zip_path = os.path.join(label_dir, file)
            print(f"🔄 Extracting {zip_path}...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)

print(f"✅ All label zips extracted to '{output_dir}'")


🔄 Extracting /content/val/labelTxt-v1.0/labelTxt.zip...
🔄 Extracting /content/val/labelTxt-v1.0/Val_Task2_gt.zip...
🔄 Extracting /content/val/labelTxt-v1.5/DOTA-v1.5_val.zip...
🔄 Extracting /content/val/labelTxt-v1.5/DOTA-v1.5_val_hbb.zip...
✅ All label zips extracted to 'val/yolov8/labels'


In [15]:
import os
from PIL import Image

# Class map (from dataset.yaml)
class_map = {
    'plane': 0, 'ship': 1, 'storage-tank': 2, 'baseball-diamond': 3,
    'tennis-court': 4, 'basketball-court': 5, 'ground-track-field': 6, 'harbor': 7,
    'bridge': 8, 'large-vehicle': 9, 'small-vehicle': 10, 'helicopter': 11,
    'roundabout': 12, 'soccer-ball-field': 13, 'swimming-pool': 14, 'container-crane': 15
}

# Paths
input_label_folder = '/content/val/yolov8/labels/valset_reclabelTxt'
image_folder = '/content/val/yolov8/images_all/images'
output_label_folder = '/content/val/yolov8/labels/labels_yolo'
os.makedirs(output_label_folder, exist_ok=True)

image_exts = ['.jpg', '.jpeg', '.png']

def find_image_path(label_filename):
    base_name, _ = os.path.splitext(label_filename)
    for ext in image_exts:
        image_path = os.path.join(image_folder, base_name + ext)
        if os.path.exists(image_path):
            return image_path
    return None

def extract_yolo_from_polygon(coords, img_w, img_h):
    xs = coords[::2]
    ys = coords[1::2]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)

    cx = ((xmin + xmax) / 2) / img_w
    cy = ((ymin + ymax) / 2) / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h

    # Clamp values between 0 and 1
    return tuple(min(max(v, 0), 1) for v in (cx, cy, w, h))

# Process label files
for label_file in os.listdir(input_label_folder):
    if not label_file.endswith('.txt'):
        continue

    image_path = find_image_path(label_file)
    if not image_path:
        print(f"⚠️ Image not found for {label_file}. Skipping.")
        continue

    try:
        with Image.open(image_path) as img:
            img_w, img_h = img.size
    except Exception as e:
        print(f"❌ Failed to open image {image_path}: {e}")
        continue

    input_path = os.path.join(input_label_folder, label_file)
    output_path = os.path.join(output_label_folder, label_file)

    with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            parts = line.strip().split()
            # Your format has 10 parts: 8 coords + class_name + "0"
            if len(parts) != 10:
                print(f"⚠️ Malformed line in {label_file}: {line.strip()}")
                continue

            try:
                coords = list(map(float, parts[:8]))
                class_name = parts[8].lower().replace('_', '-').replace(' ', '-')
                # The last part (parts[9]) seems to be a trailing zero, ignore or validate if needed

                if class_name in class_map:
                    class_id = class_map[class_name]
                else:
                    print(f"❌ Unknown class '{class_name}' in {label_file}")
                    continue

                cx, cy, w, h = extract_yolo_from_polygon(coords, img_w, img_h)
                if w == 0 or h == 0:
                    print(f"⚠️ Zero-area box in {label_file}: {line.strip()}")
                    continue

                outfile.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

            except Exception as e:
                print(f"❌ Error processing line in {label_file}: {line.strip()} — {e}")


In [16]:
import os

output_folder = '/content/val/yolov8/labels/labels_yolo'

for filename in os.listdir(output_folder):
    if not filename.endswith('.txt'):
        continue
    with open(os.path.join(output_folder, filename), 'r') as f:
        for line_num, line in enumerate(f, 1):
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Malformed line {line_num} in {filename}: {line.strip()}")
                continue
            class_id, x_c, y_c, w, h = parts
            x_c, y_c, w, h = map(float, [x_c, y_c, w, h])
            if not (0 <= x_c <= 1 and 0 <= y_c <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                print(f"Out of range values at line {line_num} in {filename}: {x_c}, {y_c}, {w}, {h}")

print("✅ Normalization check complete.")


✅ Normalization check complete.


In [17]:
import os

# Paths
image_folder = '/content/val/yolov8/images_all/images'
label_folder = '/content/val/yolov8/labels/labels_yolo'

# Accepted image extensions
image_exts = ['.jpg', '.jpeg', '.png']

# Get base names without extension
image_basenames = set()
for file in os.listdir(image_folder):
    name, ext = os.path.splitext(file)
    if ext.lower() in image_exts:
        image_basenames.add(name)

label_basenames = set()
for file in os.listdir(label_folder):
    if file.endswith('.txt'):
        name, _ = os.path.splitext(file)
        label_basenames.add(name)

# Check which images have no label
missing_labels = image_basenames - label_basenames
if missing_labels:
    print("🛑 Images without corresponding labels:")
    for name in sorted(missing_labels):
        print(f"  {name}")
else:
    print("✅ All images have corresponding labels.")

# Optional: Check for labels with no image
extra_labels = label_basenames - image_basenames
if extra_labels:
    print("\n⚠️ Labels with no corresponding images:")
    for name in sorted(extra_labels):
        print(f"  {name}")


✅ All images have corresponding labels.


# Test Dataset

In [18]:
!gdown --folder https://drive.google.com/drive/folders/1mYOf5USMGNcJRPcvRVJVV1uHEalG5RPl



Retrieving folder contents
Processing file 1fwiTNqRRen09E-O9VSpcMV2e6_d4GGVK part1.zip
Processing file 1wTwmxvPVujh1I6mCMreoKURxCUI8f-qv part2.zip
Processing file 1nQokIxSy3DEHImJribSCODTRkWlPJLE3 test_info.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1fwiTNqRRen09E-O9VSpcMV2e6_d4GGVK
From (redirected): https://drive.google.com/uc?id=1fwiTNqRRen09E-O9VSpcMV2e6_d4GGVK&confirm=t&uuid=9a96752e-6b3d-4d09-a740-dfc9dbfe021f
To: /content/images/part1.zip
100% 3.05G/3.05G [00:32<00:00, 95.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wTwmxvPVujh1I6mCMreoKURxCUI8f-qv
From (redirected): https://drive.google.com/uc?id=1wTwmxvPVujh1I6mCMreoKURxCUI8f-qv&confirm=t&uuid=76309a6e-52c2-4f96-bcfa-c2a49c3e19f9
To: /content/images/part2.zip
100% 3.61G/3.61G [00:47<00:00, 75.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nQokIxSy3DEHImJribSCOD

In [19]:
import zipfile
import os

# List of zip files to extract
zip_files = ['part1.zip','part2.zip']

# Base paths
zip_base_path = '/content/images'
extract_dir = 'test/yolov8/images_all'

# Make sure output directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_name in zip_files:
    zip_path = os.path.join(zip_base_path, zip_name)
    print(f"🔄 Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print(f"✅ All zip files extracted to '{extract_dir}'")


🔄 Extracting /content/images/part1.zip...
🔄 Extracting /content/images/part2.zip...
✅ All zip files extracted to 'test/yolov8/images_all'


In [20]:
# Train, Validation and Test Preparation

In [21]:
import os
import shutil

# Define new base dataset directory
base_dataset_dir = '/content/dataset'

# Existing paths
train_images_src = '/content/train/yolov8/images_all/images'
train_labels_src = '/content/train/yolov8/labels/labels_yolo'
val_images_src = '/content/val/yolov8/images_all/images'
val_labels_src = '/content/val/yolov8/labels/labels_yolo'
test_images_src = '/content/test/yolov8/images_all/images'

# Destination structure
train_images_dst = os.path.join(base_dataset_dir, 'train/images')
train_labels_dst = os.path.join(base_dataset_dir, 'train/labels')
val_images_dst = os.path.join(base_dataset_dir, 'val/images')
val_labels_dst = os.path.join(base_dataset_dir, 'val/labels')
test_images_dst = os.path.join(base_dataset_dir, 'test/images')

# Create destination directories
for dst_dir in [train_images_dst, train_labels_dst, val_images_dst, val_labels_dst, test_images_dst]:
    os.makedirs(dst_dir, exist_ok=True)

# Move function
def move_files(src, dst):
    for file in os.listdir(src):
        src_file = os.path.join(src, file)
        dst_file = os.path.join(dst, file)
        shutil.move(src_file, dst_file)

# Move training files
move_files(train_images_src, train_images_dst)
move_files(train_labels_src, train_labels_dst)

# Move validation files
move_files(val_images_src, val_images_dst)
move_files(val_labels_src, val_labels_dst)

# Move test files
move_files(test_images_src, test_images_dst)

print("✅ All datasets (train, val, test) have been moved into /content/dataset successfully.")


✅ All datasets (train, val, test) have been moved into /content/dataset successfully.


In [22]:
# import os
# import random
# import shutil

# # Source folders (original data)
# images_src = '/content/train/yolov8/images_all/images'
# labels_src = '/content/train/yolov8/labels/labels_yolo'

# # Destination folders following YOLOv8 dataset structure
# dataset_dir = 'dataset'
# train_images_dir = os.path.join(dataset_dir, 'train', 'images')
# train_labels_dir = os.path.join(dataset_dir, 'train', 'labels')
# val_images_dir = os.path.join(dataset_dir, 'val', 'images')
# val_labels_dir = os.path.join(dataset_dir, 'val', 'labels')

# # Create all required directories if not exist
# for folder in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
#     os.makedirs(folder, exist_ok=True)

# # List all image files from source
# all_images = [f for f in os.listdir(images_src) if f.lower().endswith(('.jpg', '.png'))]
# all_images.sort()

# # Shuffle images for randomness
# random.shuffle(all_images)

# # Split ratio (80% train, 20% val)
# split_ratio = 0.8
# split_index = int(len(all_images) * split_ratio)

# train_files = all_images[:split_index]
# val_files = all_images[split_index:]

# def copy_files(file_list, images_dest, labels_dest):
#     for fname in file_list:
#         # Copy image file
#         shutil.copy(os.path.join(images_src, fname), os.path.join(images_dest, fname))

#         # Copy corresponding label file if exists
#         label_file = fname.rsplit('.', 1)[0] + '.txt'
#         src_label_path = os.path.join(labels_src, label_file)
#         dest_label_path = os.path.join(labels_dest, label_file)
#         if os.path.exists(src_label_path):
#             shutil.copy(src_label_path, dest_label_path)
#         else:
#             print(f"⚠️ Label file not found for image: {fname}")

# # Copy train files
# copy_files(train_files, train_images_dir, train_labels_dir)

# # Copy val files
# copy_files(val_files, val_images_dir, val_labels_dir)

# print(f"✅ Dataset prepared.")
# print(f"Training samples: {len(train_files)}")
# print(f"Validation samples: {len(val_files)}")


In [23]:
%%writefile dataset.yaml
path: /content/dataset
train: /content/dataset/train/images
val: /content/dataset/val/images
test: /content/dataset/test/images
nc: 16
names:
  0: plane
  1: ship
  2: storage-tank
  3: baseball-diamond
  4: tennis-court
  5: basketball-court
  6: ground-track-field
  7: harbor
  8: bridge
  9: large-vehicle
  10: small-vehicle
  11: helicopter
  12: roundabout
  13: soccer-ball-field
  14: swimming-pool
  15: container-crane


Writing dataset.yaml


In [24]:
# import os
# import random
# import shutil

# # Source folders (original data)
# images_src = '/content/dataset/train/images'
# labels_src = '/content/dataset/train/labels'

# # Destination folders following YOLOv8 dataset structure
# dataset_dir = 'dataset'
# train_images_dir = os.path.join(dataset_dir, 'train', 'images')
# train_labels_dir = os.path.join(dataset_dir, 'train', 'labels')
# val_images_dir = os.path.join(dataset_dir, 'val', 'images')
# val_labels_dir = os.path.join(dataset_dir, 'val', 'labels')

# # Create all required directories if not exist
# for folder in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
#     os.makedirs(folder, exist_ok=True)

# # List all image files from source
# all_images = [f for f in os.listdir(images_src) if f.lower().endswith(('.jpg', '.png'))]
# all_images.sort()

# # Shuffle images for randomness
# random.shuffle(all_images)

# # Split ratio (80% train, 20% val)
# split_ratio = 0.8
# split_index = int(len(all_images) * split_ratio)

# train_files = all_images[:split_index]
# val_files = all_images[split_index:]

# def copy_files(file_list, images_dest, labels_dest):
#     for fname in file_list:
#         # Copy image file
#         shutil.copy(os.path.join(images_src, fname), os.path.join(images_dest, fname))

#         # Copy corresponding label file if exists
#         label_file = fname.rsplit('.', 1)[0] + '.txt'
#         src_label_path = os.path.join(labels_src, label_file)
#         dest_label_path = os.path.join(labels_dest, label_file)
#         if os.path.exists(src_label_path):
#             shutil.copy(src_label_path, dest_label_path)
#         else:
#             print(f"⚠️ Label file not found for image: {fname}")

# # Copy train files
# copy_files(train_files, train_images_dir, train_labels_dir)

# # Copy val files
# copy_files(val_files, val_images_dir, val_labels_dir)

# print(f"✅ Dataset prepared.")
# print(f"Training samples: {len(train_files)}")
# print(f"Validation samples: {len(val_files)}")


In [25]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [26]:
# from ultralytics import YOLO

# # Load model architecture
# model = YOLO('yolov8n.pt')

# # Train with custom settings
# model.train(
#     data='dataset.yaml',
#     epochs=10,
#     imgsz=640,
#     batch=8,              # Adjust based on your GPU memory
#     optimizer='Adam',      # 'SGD', 'Adam', 'AdamW' supported
#     lr0=0.001              # Initial learning rate
# )

In [27]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt / yolov8m.pt / yolov8l.pt depending on your needs

model.train(
    data='/content/dataset.yaml',
    epochs=15,
    imgsz=640,
    batch=8,
    project='/content/runs',
    name='yolov8-custom'
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 65.9MB/s]


Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8-custom, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]

Overriding model.yaml nc=80 with nc=16

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

      15/15      7.88G      1.626      1.305     0.9915        120        640: 100%|██████████| 177/177 [03:56<00:00,  1.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [01:02<00:00,  2.17s/it]


                   all        458      28853      0.662      0.299      0.314      0.186

15 epochs completed in 1.136 hours.
Optimizer stripped from /content/runs/yolov8-custom/weights/last.pt, 6.2MB
Optimizer stripped from /content/runs/yolov8-custom/weights/best.pt, 6.2MB

Validating /content/runs/yolov8-custom/weights/best.pt...
Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,768 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [01:14<00:00,  2.57s/it]


                   all        458      28853      0.662      0.299      0.314      0.186
                 plane         70       2531      0.694      0.499       0.52      0.263
                  ship        108       8960       0.62      0.365      0.374      0.154
          storage-tank         55       2888      0.684      0.101      0.171     0.0737
      baseball-diamond         53        214      0.621       0.28      0.308      0.156
          tennis-court         94        760      0.818      0.814      0.851      0.727
      basketball-court         35        132      0.512      0.254      0.233      0.164
    ground-track-field         70        144       0.57      0.236      0.249       0.17
                harbor        114       2090      0.629      0.451      0.471       0.23
                bridge         75        464          1          0     0.0136    0.00598
         large-vehicle        110       4387      0.649      0.684      0.668      0.399
         small-vehicl

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a80a1a35c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   

In [29]:
model = YOLO('/content/runs/yolov8-custom/weights/best.pt')  # adjust path if different

metrics = model.val(
    data='/content/dataset.yaml',
    split='test'  # uses test images
)


Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,768 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 179.9±24.4 MB/s, size: 10709.9 KB)


val: Scanning /content/dataset/test/labels... 0 images, 937 backgrounds, 0 corrupt: 100%|██████████| 937/937 [00:36<00:00, 25.63it/s]

WARNING ⚠️ val: No labels found in /content/dataset/test/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /content/dataset/test/labels.cache
WARNING ⚠️ Labels are missing or empty in /content/dataset/test/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [02:34<00:00,  2.62s/it]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:584: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:629: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:629: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_

                   all        937          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels
Speed: 0.3ms preprocess, 5.9ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/val


In [30]:
# import shutil

# # Zip the folder
# shutil.make_archive("/content/runs_zip", 'zip', "/content/runs")


'/content/runs_zip.zip'

In [31]:
# from google.colab import files
# files.download("/content/runs_zip.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# model.val()


In [ ]:
# import shutil

# # Define the source folder and destination zip file name
# source_folder = '/content/runs/detect/train'
# zip_filename = '/content/yolov8_train_results.zip'

# # Create the zip file
# shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', source_folder)

# print(f"Saved as: {zip_filename}")


In [ ]:
# import shutil

# # Define the source folder and destination zip file name
# source_folder = '/content/runs/detect/train2'
# zip_filename = '/content/yolov8_val_results.zip'

# # Create the zip file
# shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', source_folder)

# print(f"Saved as: {zip_filename}")
